In [3]:
import numpy as np
from scipy.io import loadmat
from scipy.ndimage import affine_transform
from scipy.optimize import minimize

# Load one patient's file
data = loadmat('Data/Temp Data/gz2.mat')

# Extract arrays (crop version is usually preprocessed & aligned)
left_plantar_list = data['Direct_plantar_Left_crop']
right_plantar_list = data['Direct_plantar_Right_crop']

# Convert from nested objects to numpy arrays
# each entry in the (10,1) list is an array (for 10 days)
left_plantar = [arr for arr in left_plantar_list[:,0]]
right_plantar = [arr for arr in right_plantar_list[:,0]]

def segment_foot(img, grid_rows, grid_cols, temp_threshold):
    """
    Segment the foot into a grid of small regions (patches) for localized analysis.
    Each region consists of a small number of pixels based on grid size.
    """
    h, w = img.shape
    mask = img > temp_threshold
    regions = {}
    patch_h = h // grid_rows
    patch_w = w // grid_cols

    for i in range(grid_rows):
        for j in range(grid_cols):
            r_start = i * patch_h
            r_end = min((i + 1) * patch_h, h) if i < grid_rows - 1 else h
            c_start = j * patch_w
            c_end = min((j + 1) * patch_w, w) if j < grid_cols - 1 else w
            patch_mask = mask[r_start:r_end, c_start:c_end]
            if np.any(patch_mask):
                patch_img = img[r_start:r_end, c_start:c_end][patch_mask]
                avg_temp = np.mean(patch_img) if len(patch_img) > 0 else 0.0
                key = f"region_{i}_{j}"
                regions[key] = {
                    "avg_temp": avg_temp
                }
    return regions

def analyze_day(left_img, right_img, avg_thresh=1.6, use_registration=True, use_segmentation=True, grid_rows=20, grid_cols=10, temp_threshold=20.0):
    """
    Compare left vs right thermal images for one day, with mirroring, optional registration, and optional region-based analysis.
    Focus only on average differences for whole foot and regions.
    """
    # Ensure shapes match
    assert left_img.shape == right_img.shape, "Left and Right images must be aligned"

    # Flip right image horizontally to mirror it (aligns anatomy like big toe sides)
    flipped_right = np.fliplr(right_img)

    # Optional: Register flipped_right to left_img for better alignment
    if use_registration:
        def mse_affine(params, fixed, moving):
            # Params: [scale_x, scale_y, rotation (degrees), trans_x, trans_y]
            theta = np.deg2rad(params[2])
            affine_matrix = np.array([
                [params[0] * np.cos(theta), -params[1] * np.sin(theta), params[3]],
                [params[0] * np.sin(theta), params[1] * np.cos(theta), params[4]],
                [0, 0, 1]
            ])
            warped = affine_transform(moving, affine_matrix)
            return np.mean((fixed - warped)**2)

        initial_params = [1.0, 1.0, 0.0, 0.0, 0.0]  # Start with identity
        res = minimize(mse_affine, initial_params, args=(left_img, flipped_right), method='Nelder-Mead')
        best_params = res.x

        # Apply best affine transform
        theta = np.deg2rad(best_params[2])
        affine_matrix = np.array([
            [best_params[0] * np.cos(theta), -best_params[1] * np.sin(theta), best_params[3]],
            [best_params[0] * np.sin(theta), best_params[1] * np.cos(theta), best_params[4]],
            [0, 0, 1]
        ])
        aligned_right = affine_transform(flipped_right, affine_matrix)
    else:
        aligned_right = flipped_right

    # Average asymmetry for whole foot
    avg_diff = np.abs(np.mean(left_img) - np.mean(aligned_right))

    # Start with whole foot violation
    avg_violation = avg_diff > avg_thresh

    result = {
        "avg_violation": avg_violation,
        "avg_diff": avg_diff
    }

    # Optional: Segmentation for region-based comparison
    if use_segmentation:
        left_regions = segment_foot(left_img, grid_rows, grid_cols, temp_threshold)
        aligned_right_regions = segment_foot(aligned_right, grid_rows, grid_cols, temp_threshold)

        region_diffs = {}
        violating_regions = []

        for key in set(left_regions.keys()) & set(aligned_right_regions.keys()):
            l_avg = left_regions[key]["avg_temp"]
            r_avg = aligned_right_regions[key]["avg_temp"]
            diff = np.abs(l_avg - r_avg)
            region_diffs[key] = {"avg_diff": diff}
            if diff > avg_thresh:
                avg_violation = True
                violating_regions.append(key)

        result["avg_violation"] = avg_violation
        result["region_diffs"] = region_diffs
        result["violating_regions"] = violating_regions

    return result

def analyze_over_days(left_plantar, right_plantar, days_required=5, **kwargs):
    """
    Analyze thermal images over multiple days and flag risk if violations persist for consecutive days.
    """
    assert len(left_plantar) == len(right_plantar), "Left and right lists must have the same number of days"
    
    results = []
    for left_img, right_img in zip(left_plantar, right_plantar):
        day_result = analyze_day(left_img, right_img, **kwargs)
        results.append(day_result)
    
    # Determine risk flag: True if there are at least 'days_required' consecutive days with violation
    violations = [res["avg_violation"] for res in results]
    risk_flag = False
    for i in range(len(violations) - days_required + 1):
        if all(violations[i:i+days_required]):
            risk_flag = True
            break
    
    return results, risk_flag

# Example usage (adjust grid_rows and grid_cols for smaller regions; larger values mean smaller patches)
results, risk_flag = analyze_over_days(left_plantar, right_plantar, days_required=5, grid_rows=20, grid_cols=10)

print("Daily results:")
for i, res in enumerate(results):
    print(f"Day {i+1}: AvgDiff={res['avg_diff']:.2f}, AvgFlag={res['avg_violation']}")
    if 'violating_regions' in res and res['violating_regions']:
        print(f"  Violating regions: {len(res['violating_regions'])}")

print("\nFinal Risk Flag:", risk_flag)

Daily results:
Day 1: AvgDiff=1.97, AvgFlag=True
  Violating regions: 6
Day 2: AvgDiff=1.94, AvgFlag=True
  Violating regions: 5
Day 3: AvgDiff=1.86, AvgFlag=True
  Violating regions: 8
Day 4: AvgDiff=1.89, AvgFlag=True
  Violating regions: 1
Day 5: AvgDiff=1.95, AvgFlag=True
  Violating regions: 9
Day 6: AvgDiff=2.13, AvgFlag=True
  Violating regions: 12
Day 7: AvgDiff=2.11, AvgFlag=True
  Violating regions: 11
Day 8: AvgDiff=2.02, AvgFlag=True
  Violating regions: 9
Day 9: AvgDiff=1.98, AvgFlag=True
  Violating regions: 8
Day 10: AvgDiff=1.88, AvgFlag=True
  Violating regions: 8

Final Risk Flag: True


Import stuff